# Language Translation
In this project, you’re going to take a peek into the realm of neural network machine translation.  You’ll be training a sequence to sequence model on a dataset of English and French sentences that can translate new sentences from English to French.
## Get the Data
Since translating the whole language of English to French will take lots of time to train, we have provided you with a small portion of the English corpus.

In [1]:
"""
DON'T MODIFY ANYTHING IN THIS CELL
"""
import helper
import problem_unittests as tests

source_path = 'data/small_vocab_en'
target_path = 'data/small_vocab_fr'
source_text = helper.load_data(source_path)
target_text = helper.load_data(target_path)

## Explore the Data
Play around with view_sentence_range to view different parts of the data.

In [2]:
view_sentence_range = (0, 10)

"""
DON'T MODIFY ANYTHING IN THIS CELL
"""
import numpy as np

print('Dataset Stats')
print('Roughly the number of unique words: {}'.format(len({word: None for word in source_text.split()})))

sentences = source_text.split('\n')
word_counts = [len(sentence.split()) for sentence in sentences]
print('Number of sentences: {}'.format(len(sentences)))
print('Average number of words in a sentence: {}'.format(np.average(word_counts)))

print()
print('English sentences {} to {}:'.format(*view_sentence_range))
print('\n'.join(source_text.split('\n')[view_sentence_range[0]:view_sentence_range[1]]))
print()
print('French sentences {} to {}:'.format(*view_sentence_range))
print('\n'.join(target_text.split('\n')[view_sentence_range[0]:view_sentence_range[1]]))

Dataset Stats
Roughly the number of unique words: 227
Number of sentences: 137861
Average number of words in a sentence: 13.225277634719028

English sentences 0 to 10:
new jersey is sometimes quiet during autumn , and it is snowy in april .
the united states is usually chilly during july , and it is usually freezing in november .
california is usually quiet during march , and it is usually hot in june .
the united states is sometimes mild during june , and it is cold in september .
your least liked fruit is the grape , but my least liked is the apple .
his favorite fruit is the orange , but my favorite is the grape .
paris is relaxing during december , but it is usually chilly in july .
new jersey is busy during spring , and it is never hot in march .
our least liked fruit is the lemon , but my least liked is the grape .
the united states is sometimes busy during january , and it is sometimes warm in november .

French sentences 0 to 10:
new jersey est parfois calme pendant l' automne 

## Implement Preprocessing Function
### Text to Word Ids
As you did with other RNNs, you must turn the text into a number so the computer can understand it. In the function `text_to_ids()`, you'll turn `source_text` and `target_text` from words to ids.  However, you need to add the `<EOS>` word id at the end of `target_text`.  This will help the neural network predict when the sentence should end.

You can get the `<EOS>` word id by doing:
```python
target_vocab_to_int['<EOS>']
```
You can get other word ids using `source_vocab_to_int` and `target_vocab_to_int`.

In [3]:
def text_to_ids(source_text, target_text, source_vocab_to_int, target_vocab_to_int):
    """
    Convert source and target text to proper word ids
    :param source_text: String that contains all the source text.
    :param target_text: String that contains all the target text.
    :param source_vocab_to_int: Dictionary to go from the source words to an id
    :param target_vocab_to_int: Dictionary to go from the target words to an id
    :return: A tuple of lists (source_id_text, target_id_text)
    """
    # TODO: Implement Function
    
    source_text_sentences = [sentence for sentence in source_text.split('\n')]
    
    target_text_sentences = [sentence + ' <EOS>' for sentence in target_text.split('\n')]
    
    source_sentences_id = [[source_vocab_to_int[word] for word in sentence.split()] for sentence in source_text_sentences]
    
    target_sentences_id = [[target_vocab_to_int[word] for word in sentence.split()] for sentence in target_text_sentences]
    
    return (source_sentences_id, target_sentences_id)
    
"""
DON'T MODIFY ANYTHING IN THIS CELL THAT IS BELOW THIS LINE
"""
tests.test_text_to_ids(text_to_ids)

Tests Passed


### Preprocess all the data and save it
Running the code cell below will preprocess all the data and save it to file.

In [4]:
"""
DON'T MODIFY ANYTHING IN THIS CELL
"""
helper.preprocess_and_save_data(source_path, target_path, text_to_ids)

# Check Point
This is your first checkpoint. If you ever decide to come back to this notebook or have to restart the notebook, you can start from here. The preprocessed data has been saved to disk.

In [5]:
"""
DON'T MODIFY ANYTHING IN THIS CELL
"""
import numpy as np
import helper
import problem_unittests as tests

(source_int_text, target_int_text), (source_vocab_to_int, target_vocab_to_int), _ = helper.load_preprocess()

### Check the Version of TensorFlow and Access to GPU
This will check to make sure you have the correct version of TensorFlow and access to a GPU

In [6]:
"""
DON'T MODIFY ANYTHING IN THIS CELL
"""
from distutils.version import LooseVersion
import warnings
import tensorflow as tf
from tensorflow.python.layers.core import Dense

# Check TensorFlow Version
assert LooseVersion(tf.__version__) >= LooseVersion('1.1'), 'Please use TensorFlow version 1.1 or newer'
print('TensorFlow Version: {}'.format(tf.__version__))

# Check for a GPU
if not tf.test.gpu_device_name():
    warnings.warn('No GPU found. Please use a GPU to train your neural network.')
else:
    print('Default GPU Device: {}'.format(tf.test.gpu_device_name()))

TensorFlow Version: 1.1.0
Default GPU Device: /gpu:0


## Build the Neural Network
You'll build the components necessary to build a Sequence-to-Sequence model by implementing the following functions below:
- `model_inputs`
- `process_decoder_input`
- `encoding_layer`
- `decoding_layer_train`
- `decoding_layer_infer`
- `decoding_layer`
- `seq2seq_model`

### Input
Implement the `model_inputs()` function to create TF Placeholders for the Neural Network. It should create the following placeholders:

- Input text placeholder named "input" using the TF Placeholder name parameter with rank 2.
- Targets placeholder with rank 2.
- Learning rate placeholder with rank 0.
- Keep probability placeholder named "keep_prob" using the TF Placeholder name parameter with rank 0.
- Target sequence length placeholder named "target_sequence_length" with rank 1
- Max target sequence length tensor named "max_target_len" getting its value from applying tf.reduce_max on the target_sequence_length placeholder. Rank 0.
- Source sequence length placeholder named "source_sequence_length" with rank 1

Return the placeholders in the following the tuple (input, targets, learning rate, keep probability, target sequence length, max target sequence length, source sequence length)

In [7]:
def model_inputs():
    """
    Create TF Placeholders for input, targets, learning rate, and lengths of source and target sequences.
    :return: Tuple (input, targets, learning rate, keep probability, target sequence length,
    max target sequence length, source sequence length)
    """
    # TODO: Implement Function
    inputs = tf.placeholder(tf.int32, shape=(None, None), name='input')
    targets = tf.placeholder(tf.int32, shape=(None, None))
    learning_rate = tf.placeholder(tf.float32)
    keep_prob = tf.placeholder(tf.float32, name='keep_prob')
    target_seq_lenth = tf.placeholder(tf.int32, shape=[None], name= "target_sequence_length")
    max_target_len = tf.reduce_max(target_seq_lenth, name= 'max_target_len')
    source_seq_length = tf.placeholder(tf.int32, shape=[None], name= "source_sequence_length")
    
    return (inputs, targets, learning_rate, keep_prob, target_seq_lenth, max_target_len, source_seq_length)

"""
DON'T MODIFY ANYTHING IN THIS CELL THAT IS BELOW THIS LINE
"""
tests.test_model_inputs(model_inputs)

Tests Passed


### Process Decoder Input
Implement `process_decoder_input` by removing the last word id from each batch in `target_data` and concat the GO ID to the begining of each batch.

In [8]:
def process_decoder_input(target_data, target_vocab_to_int, batch_size):
    """
    Preprocess target data for encoding
    :param target_data: Target Placehoder
    :param target_vocab_to_int: Dictionary to go from the target words to an id
    :param batch_size: Batch Size
    :return: Preprocessed target data
    """
    # TODO: Implement Function
    last_batch_id = tf.strided_slice(target_data, [0, 0], [batch_size, -1], [1, 1])
    result = tf.concat([tf.fill([batch_size, 1], target_vocab_to_int['<GO>']), last_batch_id], 1)
    return result

"""
DON'T MODIFY ANYTHING IN THIS CELL THAT IS BELOW THIS LINE
"""
tests.test_process_encoding_input(process_decoder_input)

Tests Passed


### Encoding
Implement `encoding_layer()` to create a Encoder RNN layer:
 * Embed the encoder input using [`tf.contrib.layers.embed_sequence`](https://www.tensorflow.org/api_docs/python/tf/contrib/layers/embed_sequence)
 * Construct a [stacked](https://github.com/tensorflow/tensorflow/blob/6947f65a374ebf29e74bb71e36fd82760056d82c/tensorflow/docs_src/tutorials/recurrent.md#stacking-multiple-lstms) [`tf.contrib.rnn.LSTMCell`](https://www.tensorflow.org/api_docs/python/tf/contrib/rnn/LSTMCell) wrapped in a [`tf.contrib.rnn.DropoutWrapper`](https://www.tensorflow.org/api_docs/python/tf/contrib/rnn/DropoutWrapper)
 * Pass cell and embedded input to [`tf.nn.dynamic_rnn()`](https://www.tensorflow.org/api_docs/python/tf/nn/dynamic_rnn)

In [13]:
from imp import reload
reload(tests)

def encoding_layer(rnn_inputs, rnn_size, num_layers, keep_prob, 
                   source_sequence_length, source_vocab_size, 
                   encoding_embedding_size):
    """
    Create encoding layer
    :param rnn_inputs: Inputs for the RNN
    :param rnn_size: RNN Size
    :param num_layers: Number of layers
    :param keep_prob: Dropout keep probability
    :param source_sequence_length: a list of the lengths of each sequence in the batch
    :param source_vocab_size: vocabulary size of source data
    :param encoding_embedding_size: embedding size of source data
    :return: tuple (RNN output, RNN state)
    """
    # TODO: Implement Function
    # Got help from udacity discussion forum
    # (https://discussions.udacity.com/t/help-me-for-the-encoding-layer-implementation/377704)
    
    
    n_stacked_layer = 2
    
    embed_input = tf.contrib.layers.embed_sequence(rnn_inputs, source_vocab_size, encoding_embedding_size)
    
    
    def make_cell(rnn_size):
        
        cell = tf.contrib.rnn.LSTMCell(rnn_size, initializer=tf.random_uniform_initializer(-0.1, 0.1, seed=2))
    
        drop = tf.contrib.rnn.DropoutWrapper(cell, output_keep_prob=keep_prob)
        return drop
    
    stacked_layer = tf.contrib.rnn.MultiRNNCell([make_cell(rnn_size) for _ in range(num_layers)])
    
    output, state = tf.nn.dynamic_rnn(cell=stacked_layer, inputs=embed_input, sequence_length=source_sequence_length, dtype=tf.float32)
    
    return output, state
    
"""
DON'T MODIFY ANYTHING IN THIS CELL THAT IS BELOW THIS LINE
"""
tests.test_encoding_layer(encoding_layer)

Tests Passed


### Decoding - Training
Create a training decoding layer:
* Create a [`tf.contrib.seq2seq.TrainingHelper`](https://www.tensorflow.org/api_docs/python/tf/contrib/seq2seq/TrainingHelper) 
* Create a [`tf.contrib.seq2seq.BasicDecoder`](https://www.tensorflow.org/api_docs/python/tf/contrib/seq2seq/BasicDecoder)
* Obtain the decoder outputs from [`tf.contrib.seq2seq.dynamic_decode`](https://www.tensorflow.org/api_docs/python/tf/contrib/seq2seq/dynamic_decode)

In [17]:

def decoding_layer_train(encoder_state, dec_cell, dec_embed_input, 
                         target_sequence_length, max_summary_length, 
                         output_layer, keep_prob):
    """
    Create a decoding layer for training
    :param encoder_state: Encoder State
    :param dec_cell: Decoder RNN Cell
    :param dec_embed_input: Decoder embedded input
    :param target_sequence_length: The lengths of each sequence in the target batch
    :param max_summary_length: The length of the longest sequence in the batch
    :param output_layer: Function to apply the output layer
    :param keep_prob: Dropout keep probability
    :return: BasicDecoderOutput containing training logits and sample_id
    """
    # TODO: Implement Function
    helper = tf.contrib.seq2seq.TrainingHelper(dec_embed_input, target_sequence_length)
    decoder = tf.contrib.seq2seq.BasicDecoder(dec_cell, helper, encoder_state, output_layer)
    f_output, _= tf.contrib.seq2seq.dynamic_decode(decoder,maximum_iterations=max_summary_length)
    return f_output


"""
DON'T MODIFY ANYTHING IN THIS CELL THAT IS BELOW THIS LINE
"""
tests.test_decoding_layer_train(decoding_layer_train)

Tests Passed


### Decoding - Inference
Create inference decoder:
* Create a [`tf.contrib.seq2seq.GreedyEmbeddingHelper`](https://www.tensorflow.org/api_docs/python/tf/contrib/seq2seq/GreedyEmbeddingHelper)
* Create a [`tf.contrib.seq2seq.BasicDecoder`](https://www.tensorflow.org/api_docs/python/tf/contrib/seq2seq/BasicDecoder)
* Obtain the decoder outputs from [`tf.contrib.seq2seq.dynamic_decode`](https://www.tensorflow.org/api_docs/python/tf/contrib/seq2seq/dynamic_decode)

In [20]:
def decoding_layer_infer(encoder_state, dec_cell, dec_embeddings, start_of_sequence_id,
                         end_of_sequence_id, max_target_sequence_length,
                         vocab_size, output_layer, batch_size, keep_prob):
    """
    Create a decoding layer for inference
    :param encoder_state: Encoder state
    :param dec_cell: Decoder RNN Cell
    :param dec_embeddings: Decoder embeddings
    :param start_of_sequence_id: GO ID
    :param end_of_sequence_id: EOS Id
    :param max_target_sequence_length: Maximum length of target sequences
    :param vocab_size: Size of decoder/target vocabulary
    :param decoding_scope: TenorFlow Variable Scope for decoding
    :param output_layer: Function to apply the output layer
    :param batch_size: Batch size
    :param keep_prob: Dropout keep probability
    :return: BasicDecoderOutput containing inference logits and sample_id
    """
    # TODO: Implement Function
    # got help from an online github repository and udacity discussion board
    
    start_ids = tf.tile([start_of_sequence_id], [batch_size])
    
    embedding_helper = tf.contrib.seq2seq.GreedyEmbeddingHelper(dec_embeddings, start_ids, end_of_sequence_id)
    
    basic_decoder = tf.contrib.seq2seq.BasicDecoder(dec_cell, embedding_helper, encoder_state, output_layer)
    
    f_output, _= tf.contrib.seq2seq.dynamic_decode(basic_decoder,maximum_iterations=max_target_sequence_length)
    
    return f_output



"""
DON'T MODIFY ANYTHING IN THIS CELL THAT IS BELOW THIS LINE
"""
tests.test_decoding_layer_infer(decoding_layer_infer)

Tests Passed


### Build the Decoding Layer
Implement `decoding_layer()` to create a Decoder RNN layer.

* Embed the target sequences
* Construct the decoder LSTM cell (just like you constructed the encoder cell above)
* Create an output layer to map the outputs of the decoder to the elements of our vocabulary
* Use the your `decoding_layer_train(encoder_state, dec_cell, dec_embed_input, target_sequence_length, max_target_sequence_length, output_layer, keep_prob)` function to get the training logits.
* Use your `decoding_layer_infer(encoder_state, dec_cell, dec_embeddings, start_of_sequence_id, end_of_sequence_id, max_target_sequence_length, vocab_size, output_layer, batch_size, keep_prob)` function to get the inference logits.

Note: You'll need to use [tf.variable_scope](https://www.tensorflow.org/api_docs/python/tf/variable_scope) to share variables between training and inference.

In [45]:
def decoding_layer(dec_input, encoder_state,
                   target_sequence_length, max_target_sequence_length,
                   rnn_size,
                   num_layers, target_vocab_to_int, target_vocab_size,
                   batch_size, keep_prob, decoding_embedding_size):
    """
    Create decoding layer
    :param dec_input: Decoder input
    :param encoder_state: Encoder state
    :param target_sequence_length: The lengths of each sequence in the target batch
    :param max_target_sequence_length: Maximum length of target sequences
    :param rnn_size: RNN Size
    :param num_layers: Number of layers
    :param target_vocab_to_int: Dictionary to go from the target words to an id
    :param target_vocab_size: Size of target vocabulary
    :param batch_size: The size of the batch
    :param keep_prob: Dropout keep probability
    :param decoding_embedding_size: Decoding embedding size
    :return: Tuple of (Training BasicDecoderOutput, Inference BasicDecoderOutput)
    """
    # TODO: Implement Function
    def build_cell(rnn_size, keep_prob):
        lstm = tf.contrib.rnn.LSTMCell(rnn_size)
        lstm_drop = tf.contrib.rnn.DropoutWrapper(lstm, output_keep_prob=keep_prob)
        return lstm_drop
    
    stacking_lstm = tf.contrib.rnn.MultiRNNCell([build_cell(rnn_size, keep_prob) for _ in range(num_layers)])
    
    dec_embeddings = tf.Variable(tf.random_uniform([target_vocab_size, decoding_embedding_size]))
    dec_embed_input = tf.nn.embedding_lookup(dec_embeddings, dec_input)

    dense_layer = Dense(target_vocab_size,kernel_initializer = tf.truncated_normal_initializer(mean = 0.0, stddev=0.1))
    
    with tf.variable_scope("decode") as scope:
        train_decoder_output = decoding_layer_train(
            encoder_state, stacking_lstm, dec_embed_input, 
            target_sequence_length, max_target_sequence_length, 
            dense_layer, keep_prob)
        
        scope.reuse_variables()
        
        inference_decoder_output = decoding_layer_infer(
            encoder_state, stacking_lstm, dec_embeddings, 
            target_vocab_to_int['<GO>'], target_vocab_to_int['<EOS>'], 
            max_target_sequence_length, target_vocab_size, 
            dense_layer, batch_size, keep_prob)
    
    return train_decoder_output, inference_decoder_output

"""
DON'T MODIFY ANYTHING IN THIS CELL THAT IS BELOW THIS LINE
"""
tests.test_decoding_layer(decoding_layer)

Tests Passed


### Build the Neural Network
Apply the functions you implemented above to:

- Encode the input using your `encoding_layer(rnn_inputs, rnn_size, num_layers, keep_prob,  source_sequence_length, source_vocab_size, encoding_embedding_size)`.
- Process target data using your `process_decoder_input(target_data, target_vocab_to_int, batch_size)` function.
- Decode the encoded input using your `decoding_layer(dec_input, enc_state, target_sequence_length, max_target_sentence_length, rnn_size, num_layers, target_vocab_to_int, target_vocab_size, batch_size, keep_prob, dec_embedding_size)` function.

In [47]:
def seq2seq_model(input_data, target_data, keep_prob, batch_size,
                  source_sequence_length, target_sequence_length,
                  max_target_sentence_length,
                  source_vocab_size, target_vocab_size,
                  enc_embedding_size, dec_embedding_size,
                  rnn_size, num_layers, target_vocab_to_int):
    """
    Build the Sequence-to-Sequence part of the neural network
    :param input_data: Input placeholder
    :param target_data: Target placeholder
    :param keep_prob: Dropout keep probability placeholder
    :param batch_size: Batch Size
    :param source_sequence_length: Sequence Lengths of source sequences in the batch
    :param target_sequence_length: Sequence Lengths of target sequences in the batch
    :param source_vocab_size: Source vocabulary size
    :param target_vocab_size: Target vocabulary size
    :param enc_embedding_size: Decoder embedding size
    :param dec_embedding_size: Encoder embedding size
    :param rnn_size: RNN Size
    :param num_layers: Number of layers
    :param target_vocab_to_int: Dictionary to go from the target words to an id
    :return: Tuple of (Training BasicDecoderOutput, Inference BasicDecoderOutput)
    """
    # TODO: Implement Function
    # ran into a couple of errors took help from
    # (https://discussions.udacity.com/t/def-seq2seq-model/277476)
    
    output, state = encoding_layer(input_data, rnn_size, num_layers, keep_prob, 
                   source_sequence_length, source_vocab_size, 
                   enc_embedding_size)
    
    processed_input = process_decoder_input(target_data, target_vocab_to_int, batch_size)
    
    training_decoder_output, inference_decoder_output = decoding_layer(processed_input, state,
                   target_sequence_length, max_target_sentence_length,
                   rnn_size, num_layers, target_vocab_to_int, target_vocab_size,
                   batch_size, keep_prob, dec_embedding_size)
    
    return training_decoder_output, inference_decoder_output


"""
DON'T MODIFY ANYTHING IN THIS CELL THAT IS BELOW THIS LINE
"""
tests.test_seq2seq_model(seq2seq_model)

Tests Passed


## Neural Network Training
### Hyperparameters
Tune the following parameters:

- Set `epochs` to the number of epochs.
- Set `batch_size` to the batch size.
- Set `rnn_size` to the size of the RNNs.
- Set `num_layers` to the number of layers.
- Set `encoding_embedding_size` to the size of the embedding for the encoder.
- Set `decoding_embedding_size` to the size of the embedding for the decoder.
- Set `learning_rate` to the learning rate.
- Set `keep_probability` to the Dropout keep probability
- Set `display_step` to state how many steps between each debug output statement

In [61]:
# Number of Epochs
epochs = 5
# Batch Size
batch_size = 512
# RNN Size
rnn_size = 512
# Number of Layers
num_layers = 5
# Embedding Size
encoding_embedding_size = 128
decoding_embedding_size = 128
# Learning Rate
learning_rate = 0.001
# Dropout Keep Probability
keep_probability = 0.75
display_step = True

### Build the Graph
Build the graph using the neural network you implemented.

In [62]:
"""
DON'T MODIFY ANYTHING IN THIS CELL
"""
save_path = 'checkpoints/dev'
(source_int_text, target_int_text), (source_vocab_to_int, target_vocab_to_int), _ = helper.load_preprocess()
max_target_sentence_length = max([len(sentence) for sentence in source_int_text])

train_graph = tf.Graph()
with train_graph.as_default():
    input_data, targets, lr, keep_prob, target_sequence_length, max_target_sequence_length, source_sequence_length = model_inputs()

    #sequence_length = tf.placeholder_with_default(max_target_sentence_length, None, name='sequence_length')
    input_shape = tf.shape(input_data)

    train_logits, inference_logits = seq2seq_model(tf.reverse(input_data, [-1]),
                                                   targets,
                                                   keep_prob,
                                                   batch_size,
                                                   source_sequence_length,
                                                   target_sequence_length,
                                                   max_target_sequence_length,
                                                   len(source_vocab_to_int),
                                                   len(target_vocab_to_int),
                                                   encoding_embedding_size,
                                                   decoding_embedding_size,
                                                   rnn_size,
                                                   num_layers,
                                                   target_vocab_to_int)


    training_logits = tf.identity(train_logits.rnn_output, name='logits')
    inference_logits = tf.identity(inference_logits.sample_id, name='predictions')

    masks = tf.sequence_mask(target_sequence_length, max_target_sequence_length, dtype=tf.float32, name='masks')

    with tf.name_scope("optimization"):
        # Loss function
        cost = tf.contrib.seq2seq.sequence_loss(
            training_logits,
            targets,
            masks)

        # Optimizer
        optimizer = tf.train.AdamOptimizer(lr)

        # Gradient Clipping
        gradients = optimizer.compute_gradients(cost)
        capped_gradients = [(tf.clip_by_value(grad, -1., 1.), var) for grad, var in gradients if grad is not None]
        train_op = optimizer.apply_gradients(capped_gradients)


Batch and pad the source and target sequences

In [63]:
"""
DON'T MODIFY ANYTHING IN THIS CELL
"""
def pad_sentence_batch(sentence_batch, pad_int):
    """Pad sentences with <PAD> so that each sentence of a batch has the same length"""
    max_sentence = max([len(sentence) for sentence in sentence_batch])
    return [sentence + [pad_int] * (max_sentence - len(sentence)) for sentence in sentence_batch]


def get_batches(sources, targets, batch_size, source_pad_int, target_pad_int):
    """Batch targets, sources, and the lengths of their sentences together"""
    for batch_i in range(0, len(sources)//batch_size):
        start_i = batch_i * batch_size

        # Slice the right amount for the batch
        sources_batch = sources[start_i:start_i + batch_size]
        targets_batch = targets[start_i:start_i + batch_size]

        # Pad
        pad_sources_batch = np.array(pad_sentence_batch(sources_batch, source_pad_int))
        pad_targets_batch = np.array(pad_sentence_batch(targets_batch, target_pad_int))

        # Need the lengths for the _lengths parameters
        pad_targets_lengths = []
        for target in pad_targets_batch:
            pad_targets_lengths.append(len(target))

        pad_source_lengths = []
        for source in pad_sources_batch:
            pad_source_lengths.append(len(source))

        yield pad_sources_batch, pad_targets_batch, pad_source_lengths, pad_targets_lengths


### Train
Train the neural network on the preprocessed data. If you have a hard time getting a good loss, check the forms to see if anyone is having the same problem.

In [64]:
"""
DON'T MODIFY ANYTHING IN THIS CELL
"""
def get_accuracy(target, logits):
    """
    Calculate accuracy
    """
    max_seq = max(target.shape[1], logits.shape[1])
    if max_seq - target.shape[1]:
        target = np.pad(
            target,
            [(0,0),(0,max_seq - target.shape[1])],
            'constant')
    if max_seq - logits.shape[1]:
        logits = np.pad(
            logits,
            [(0,0),(0,max_seq - logits.shape[1])],
            'constant')

    return np.mean(np.equal(target, logits))

# Split data to training and validation sets
train_source = source_int_text[batch_size:]
train_target = target_int_text[batch_size:]
valid_source = source_int_text[:batch_size]
valid_target = target_int_text[:batch_size]
(valid_sources_batch, valid_targets_batch, valid_sources_lengths, valid_targets_lengths ) = next(get_batches(valid_source,
                                                                                                             valid_target,
                                                                                                             batch_size,
                                                                                                             source_vocab_to_int['<PAD>'],
                                                                                                             target_vocab_to_int['<PAD>']))                                                                                                  
with tf.Session(graph=train_graph) as sess:
    sess.run(tf.global_variables_initializer())

    for epoch_i in range(epochs):
        for batch_i, (source_batch, target_batch, sources_lengths, targets_lengths) in enumerate(
                get_batches(train_source, train_target, batch_size,
                            source_vocab_to_int['<PAD>'],
                            target_vocab_to_int['<PAD>'])):

            _, loss = sess.run(
                [train_op, cost],
                {input_data: source_batch,
                 targets: target_batch,
                 lr: learning_rate,
                 target_sequence_length: targets_lengths,
                 source_sequence_length: sources_lengths,
                 keep_prob: keep_probability})


            if batch_i % display_step == 0 and batch_i > 0:


                batch_train_logits = sess.run(
                    inference_logits,
                    {input_data: source_batch,
                     source_sequence_length: sources_lengths,
                     target_sequence_length: targets_lengths,
                     keep_prob: 1.0})


                batch_valid_logits = sess.run(
                    inference_logits,
                    {input_data: valid_sources_batch,
                     source_sequence_length: valid_sources_lengths,
                     target_sequence_length: valid_targets_lengths,
                     keep_prob: 1.0})

                train_acc = get_accuracy(target_batch, batch_train_logits)

                valid_acc = get_accuracy(valid_targets_batch, batch_valid_logits)

                print('Epoch {:>3} Batch {:>4}/{} - Train Accuracy: {:>6.4f}, Validation Accuracy: {:>6.4f}, Loss: {:>6.4f}'
                      .format(epoch_i, batch_i, len(source_int_text) // batch_size, train_acc, valid_acc, loss))

    # Save Model
    saver = tf.train.Saver()
    saver.save(sess, save_path)
    print('Model Trained and Saved')

Epoch   0 Batch    1/269 - Train Accuracy: 0.2721, Validation Accuracy: 0.3465, Loss: 5.3380
Epoch   0 Batch    2/269 - Train Accuracy: 0.0640, Validation Accuracy: 0.1032, Loss: 4.6117
Epoch   0 Batch    3/269 - Train Accuracy: 0.2587, Validation Accuracy: 0.3249, Loss: 4.6424
Epoch   0 Batch    4/269 - Train Accuracy: 0.2616, Validation Accuracy: 0.3339, Loss: 4.1543
Epoch   0 Batch    5/269 - Train Accuracy: 0.2796, Validation Accuracy: 0.3489, Loss: 3.8045
Epoch   0 Batch    6/269 - Train Accuracy: 0.3150, Validation Accuracy: 0.3448, Loss: 3.4477
Epoch   0 Batch    7/269 - Train Accuracy: 0.3313, Validation Accuracy: 0.3632, Loss: 3.3684
Epoch   0 Batch    8/269 - Train Accuracy: 0.3252, Validation Accuracy: 0.3873, Loss: 3.4431
Epoch   0 Batch    9/269 - Train Accuracy: 0.3362, Validation Accuracy: 0.3764, Loss: 3.1808
Epoch   0 Batch   10/269 - Train Accuracy: 0.3218, Validation Accuracy: 0.3864, Loss: 3.2590
Epoch   0 Batch   11/269 - Train Accuracy: 0.3634, Validation Accuracy

Epoch   0 Batch   90/269 - Train Accuracy: 0.4449, Validation Accuracy: 0.5070, Loss: 1.8129
Epoch   0 Batch   91/269 - Train Accuracy: 0.4864, Validation Accuracy: 0.5088, Loss: 1.7075
Epoch   0 Batch   92/269 - Train Accuracy: 0.4873, Validation Accuracy: 0.5139, Loss: 1.7025
Epoch   0 Batch   93/269 - Train Accuracy: 0.5304, Validation Accuracy: 0.5303, Loss: 1.6176
Epoch   0 Batch   94/269 - Train Accuracy: 0.5050, Validation Accuracy: 0.5251, Loss: 1.7447
Epoch   0 Batch   95/269 - Train Accuracy: 0.4831, Validation Accuracy: 0.5100, Loss: 1.6480
Epoch   0 Batch   96/269 - Train Accuracy: 0.4961, Validation Accuracy: 0.5152, Loss: 1.7033
Epoch   0 Batch   97/269 - Train Accuracy: 0.4927, Validation Accuracy: 0.5137, Loss: 1.6720
Epoch   0 Batch   98/269 - Train Accuracy: 0.4974, Validation Accuracy: 0.5120, Loss: 1.6529
Epoch   0 Batch   99/269 - Train Accuracy: 0.4739, Validation Accuracy: 0.5193, Loss: 1.7354
Epoch   0 Batch  100/269 - Train Accuracy: 0.5272, Validation Accuracy

Epoch   0 Batch  179/269 - Train Accuracy: 0.4676, Validation Accuracy: 0.4953, Loss: 1.4876
Epoch   0 Batch  180/269 - Train Accuracy: 0.4335, Validation Accuracy: 0.4687, Loss: 1.3921
Epoch   0 Batch  181/269 - Train Accuracy: 0.5118, Validation Accuracy: 0.5335, Loss: 1.5598
Epoch   0 Batch  182/269 - Train Accuracy: 0.5305, Validation Accuracy: 0.5404, Loss: 1.4214
Epoch   0 Batch  183/269 - Train Accuracy: 0.5617, Validation Accuracy: 0.5109, Loss: 1.1562
Epoch   0 Batch  184/269 - Train Accuracy: 0.4549, Validation Accuracy: 0.4986, Loss: 1.4169
Epoch   0 Batch  185/269 - Train Accuracy: 0.5033, Validation Accuracy: 0.5152, Loss: 1.3912
Epoch   0 Batch  186/269 - Train Accuracy: 0.4704, Validation Accuracy: 0.5186, Loss: 1.3503
Epoch   0 Batch  187/269 - Train Accuracy: 0.5139, Validation Accuracy: 0.5268, Loss: 1.2824
Epoch   0 Batch  188/269 - Train Accuracy: 0.5102, Validation Accuracy: 0.5258, Loss: 1.2649
Epoch   0 Batch  189/269 - Train Accuracy: 0.5255, Validation Accuracy

Epoch   1 Batch    1/269 - Train Accuracy: 0.5642, Validation Accuracy: 0.5796, Loss: 1.0108
Epoch   1 Batch    2/269 - Train Accuracy: 0.5561, Validation Accuracy: 0.5797, Loss: 1.0266
Epoch   1 Batch    3/269 - Train Accuracy: 0.5587, Validation Accuracy: 0.5770, Loss: 1.0253
Epoch   1 Batch    4/269 - Train Accuracy: 0.5555, Validation Accuracy: 0.5866, Loss: 1.0110
Epoch   1 Batch    5/269 - Train Accuracy: 0.5604, Validation Accuracy: 0.5903, Loss: 0.9949
Epoch   1 Batch    6/269 - Train Accuracy: 0.5903, Validation Accuracy: 0.5888, Loss: 0.9258
Epoch   1 Batch    7/269 - Train Accuracy: 0.5856, Validation Accuracy: 0.5903, Loss: 0.9719
Epoch   1 Batch    8/269 - Train Accuracy: 0.5627, Validation Accuracy: 0.5948, Loss: 1.0445
Epoch   1 Batch    9/269 - Train Accuracy: 0.5712, Validation Accuracy: 0.5942, Loss: 0.9908
Epoch   1 Batch   10/269 - Train Accuracy: 0.5811, Validation Accuracy: 0.5961, Loss: 0.9754
Epoch   1 Batch   11/269 - Train Accuracy: 0.5742, Validation Accuracy

Epoch   1 Batch   90/269 - Train Accuracy: 0.5698, Validation Accuracy: 0.6058, Loss: 0.9018
Epoch   1 Batch   91/269 - Train Accuracy: 0.6036, Validation Accuracy: 0.6071, Loss: 0.8222
Epoch   1 Batch   92/269 - Train Accuracy: 0.6077, Validation Accuracy: 0.6159, Loss: 0.8511
Epoch   1 Batch   93/269 - Train Accuracy: 0.6235, Validation Accuracy: 0.6217, Loss: 0.8020
Epoch   1 Batch   94/269 - Train Accuracy: 0.6083, Validation Accuracy: 0.6129, Loss: 0.8713
Epoch   1 Batch   95/269 - Train Accuracy: 0.6060, Validation Accuracy: 0.6127, Loss: 0.8557
Epoch   1 Batch   96/269 - Train Accuracy: 0.6127, Validation Accuracy: 0.6042, Loss: 0.8486
Epoch   1 Batch   97/269 - Train Accuracy: 0.5980, Validation Accuracy: 0.6018, Loss: 0.8521
Epoch   1 Batch   98/269 - Train Accuracy: 0.5985, Validation Accuracy: 0.6123, Loss: 0.8569
Epoch   1 Batch   99/269 - Train Accuracy: 0.5935, Validation Accuracy: 0.6114, Loss: 0.8918
Epoch   1 Batch  100/269 - Train Accuracy: 0.6341, Validation Accuracy

...

Epoch   3 Batch  179/269 - Train Accuracy: 0.7978, Validation Accuracy: 0.8235, Loss: 0.2091
Epoch   3 Batch  180/269 - Train Accuracy: 0.8397, Validation Accuracy: 0.8228, Loss: 0.2085
Epoch   3 Batch  181/269 - Train Accuracy: 0.8241, Validation Accuracy: 0.8181, Loss: 0.2149
Epoch   3 Batch  182/269 - Train Accuracy: 0.8291, Validation Accuracy: 0.8135, Loss: 0.2122
Epoch   3 Batch  183/269 - Train Accuracy: 0.8576, Validation Accuracy: 0.8258, Loss: 0.1794
Epoch   3 Batch  184/269 - Train Accuracy: 0.8566, Validation Accuracy: 0.8271, Loss: 0.2177
Epoch   3 Batch  185/269 - Train Accuracy: 0.8443, Validation Accuracy: 0.8228, Loss: 0.2087
Epoch   3 Batch  186/269 - Train Accuracy: 0.8425, Validation Accuracy: 0.8300, Loss: 0.2138
Epoch   3 Batch  187/269 - Train Accuracy: 0.8224, Validation Accuracy: 0.8354, Loss: 0.2039
Epoch   3 Batch  188/269 - Train Accuracy: 0.8333, Validation Accuracy: 0.8302, Loss: 0.2159
Epoch   3 Batch  189/269 - Train Accuracy: 0.8365, Validation Accuracy

Epoch   4 Batch    1/269 - Train Accuracy: 0.8639, Validation Accuracy: 0.8340, Loss: 0.1842
Epoch   4 Batch    2/269 - Train Accuracy: 0.8298, Validation Accuracy: 0.8191, Loss: 0.1790
Epoch   4 Batch    3/269 - Train Accuracy: 0.8466, Validation Accuracy: 0.8283, Loss: 0.1920
Epoch   4 Batch    4/269 - Train Accuracy: 0.8452, Validation Accuracy: 0.8335, Loss: 0.1828
Epoch   4 Batch    5/269 - Train Accuracy: 0.8445, Validation Accuracy: 0.8384, Loss: 0.1819
Epoch   4 Batch    6/269 - Train Accuracy: 0.8595, Validation Accuracy: 0.8359, Loss: 0.1723
Epoch   4 Batch    7/269 - Train Accuracy: 0.8572, Validation Accuracy: 0.8309, Loss: 0.1791
Epoch   4 Batch    8/269 - Train Accuracy: 0.8373, Validation Accuracy: 0.8382, Loss: 0.1851
Epoch   4 Batch    9/269 - Train Accuracy: 0.8404, Validation Accuracy: 0.8362, Loss: 0.1846
Epoch   4 Batch   10/269 - Train Accuracy: 0.8528, Validation Accuracy: 0.8484, Loss: 0.1803
Epoch   4 Batch   11/269 - Train Accuracy: 0.8418, Validation Accuracy

Epoch   4 Batch   90/269 - Train Accuracy: 0.8497, Validation Accuracy: 0.8620, Loss: 0.1579
Epoch   4 Batch   91/269 - Train Accuracy: 0.8815, Validation Accuracy: 0.8596, Loss: 0.1468
Epoch   4 Batch   92/269 - Train Accuracy: 0.8902, Validation Accuracy: 0.8569, Loss: 0.1416
Epoch   4 Batch   93/269 - Train Accuracy: 0.8804, Validation Accuracy: 0.8602, Loss: 0.1480
Epoch   4 Batch   94/269 - Train Accuracy: 0.8670, Validation Accuracy: 0.8605, Loss: 0.1608
Epoch   4 Batch   95/269 - Train Accuracy: 0.8797, Validation Accuracy: 0.8666, Loss: 0.1508
Epoch   4 Batch   96/269 - Train Accuracy: 0.8701, Validation Accuracy: 0.8715, Loss: 0.1595
Epoch   4 Batch   97/269 - Train Accuracy: 0.8627, Validation Accuracy: 0.8624, Loss: 0.1471
Epoch   4 Batch   98/269 - Train Accuracy: 0.8786, Validation Accuracy: 0.8613, Loss: 0.1523
Epoch   4 Batch   99/269 - Train Accuracy: 0.8614, Validation Accuracy: 0.8680, Loss: 0.1583
Epoch   4 Batch  100/269 - Train Accuracy: 0.8717, Validation Accuracy

Epoch   4 Batch  179/269 - Train Accuracy: 0.8598, Validation Accuracy: 0.8746, Loss: 0.1353
Epoch   4 Batch  180/269 - Train Accuracy: 0.8995, Validation Accuracy: 0.8707, Loss: 0.1358
Epoch   4 Batch  181/269 - Train Accuracy: 0.8729, Validation Accuracy: 0.8651, Loss: 0.1385
Epoch   4 Batch  182/269 - Train Accuracy: 0.8641, Validation Accuracy: 0.8590, Loss: 0.1328
Epoch   4 Batch  183/269 - Train Accuracy: 0.8914, Validation Accuracy: 0.8730, Loss: 0.1199
Epoch   4 Batch  184/269 - Train Accuracy: 0.8884, Validation Accuracy: 0.8722, Loss: 0.1343
Epoch   4 Batch  185/269 - Train Accuracy: 0.8820, Validation Accuracy: 0.8683, Loss: 0.1354
Epoch   4 Batch  186/269 - Train Accuracy: 0.8909, Validation Accuracy: 0.8716, Loss: 0.1301
Epoch   4 Batch  187/269 - Train Accuracy: 0.8665, Validation Accuracy: 0.8555, Loss: 0.1291
Epoch   4 Batch  188/269 - Train Accuracy: 0.8818, Validation Accuracy: 0.8690, Loss: 0.1428
Epoch   4 Batch  189/269 - Train Accuracy: 0.8780, Validation Accuracy

Model Trained and Saved


### Save Parameters
Save the `batch_size` and `save_path` parameters for inference.

In [65]:
"""
DON'T MODIFY ANYTHING IN THIS CELL
"""
# Save parameters for checkpoint
helper.save_params(save_path)

# Checkpoint

In [66]:
"""
DON'T MODIFY ANYTHING IN THIS CELL
"""
import tensorflow as tf
import numpy as np
import helper
import problem_unittests as tests

_, (source_vocab_to_int, target_vocab_to_int), (source_int_to_vocab, target_int_to_vocab) = helper.load_preprocess()
load_path = helper.load_params()

## Sentence to Sequence
To feed a sentence into the model for translation, you first need to preprocess it.  Implement the function `sentence_to_seq()` to preprocess new sentences.

- Convert the sentence to lowercase
- Convert words into ids using `vocab_to_int`
 - Convert words not in the vocabulary, to the `<UNK>` word id.

In [67]:
def sentence_to_seq(sentence, vocab_to_int):
    """
    Convert a sentence to a sequence of ids
    :param sentence: String
    :param vocab_to_int: Dictionary to go from the words to an id
    :return: List of word ids
    """
    # TODO: Implement Function
    words = sentence.lower().split()
    word_id_list = [vocab_to_int.get(word, vocab_to_int['<UNK>']) for word in words]
    return word_id_list


"""
DON'T MODIFY ANYTHING IN THIS CELL THAT IS BELOW THIS LINE
"""
tests.test_sentence_to_seq(sentence_to_seq)

Tests Passed


## Translate
This will translate `translate_sentence` from English to French.

In [68]:
translate_sentence = 'he saw a old yellow truck .'


"""
DON'T MODIFY ANYTHING IN THIS CELL
"""
translate_sentence = sentence_to_seq(translate_sentence, source_vocab_to_int)

loaded_graph = tf.Graph()
with tf.Session(graph=loaded_graph) as sess:
    # Load saved model
    loader = tf.train.import_meta_graph(load_path + '.meta')
    loader.restore(sess, load_path)

    input_data = loaded_graph.get_tensor_by_name('input:0')
    logits = loaded_graph.get_tensor_by_name('predictions:0')
    target_sequence_length = loaded_graph.get_tensor_by_name('target_sequence_length:0')
    source_sequence_length = loaded_graph.get_tensor_by_name('source_sequence_length:0')
    keep_prob = loaded_graph.get_tensor_by_name('keep_prob:0')

    translate_logits = sess.run(logits, {input_data: [translate_sentence]*batch_size,
                                         target_sequence_length: [len(translate_sentence)*2]*batch_size,
                                         source_sequence_length: [len(translate_sentence)]*batch_size,
                                         keep_prob: 1.0})[0]

print('Input')
print('  Word Ids:      {}'.format([i for i in translate_sentence]))
print('  English Words: {}'.format([source_int_to_vocab[i] for i in translate_sentence]))

print('\nPrediction')
print('  Word Ids:      {}'.format([i for i in translate_logits]))
print('  French Words: {}'.format(" ".join([target_int_to_vocab[i] for i in translate_logits])))


INFO:tensorflow:Restoring parameters from checkpoints/dev
Input
  Word Ids:      [31, 134, 22, 206, 158, 111, 133]
  English Words: ['he', 'saw', 'a', 'old', 'yellow', 'truck', '.']

Prediction
  Word Ids:      [98, 138, 263, 321, 239, 222, 1]
  French Words: mon fruit préféré est moins . <EOS>


## Imperfect Translation
You might notice that some sentences translate better than others.  Since the dataset you're using only has a vocabulary of 227 English words of the thousands that you use, you're only going to see good results using these words.  For this project, you don't need a perfect translation. However, if you want to create a better translation model, you'll need better data.

You can train on the [WMT10 French-English corpus](http://www.statmt.org/wmt10/training-giga-fren.tar).  This dataset has more vocabulary and richer in topics discussed.  However, this will take you days to train, so make sure you've a GPU and the neural network is performing well on dataset we provided.  Just make sure you play with the WMT10 corpus after you've submitted this project.
## Submitting This Project
When submitting this project, make sure to run all the cells before saving the notebook. Save the notebook file as "dlnd_language_translation.ipynb" and save it as a HTML file under "File" -> "Download as". Include the "helper.py" and "problem_unittests.py" files in your submission.